##### 问题
你想根据一组语法规则解析文本并执行命令，或者构造一个代表输入的抽象语法树。如果语法简单，你可以自己写这个解析器，而不是使用一些框架。

##### 解决方案
在这个问题中，我们集中讨论根据特别语法去解析文本的问题。为了这样做，你先要以BNF或者EBNF指定一个标准语法。比如，一个简单数学表达式语法可能像下面这样：

或者，以EBNF形式：

在EBNF中，被包含在`{...}*`中的规则是可选的。`*`代表0次或多次重复（跟正则表达式中意义是一样的）。  
现在，如果你对BNF的工作机制还不是很明白的话，就把它当做是一组左右符号可相互替换的规则。一般来讲，解析的原理就是你利用BNF完成多个替换和扩展以匹配输入文本和语法规则。为了演示，假设你正在解析形如`3 + 4 * 5`的表达式。这个表达式要通过使用2.18节中介绍的技术为一组令牌流。结果可能是像下列这样的令牌序列：

在此基础上，解析动作会试着通过替换操作匹配语法到输入令牌：

下面所有的解析可能需要花点时间弄明白，但是他们原理都是查找输入并试着去匹配语法规则。第一个输入令牌是NUM，因此替换首先会匹配那个部分。一旦匹配成功，就会进入下一个令牌+，以此类推。当已经确定不能撇皮下一个令牌的时候，右边的部分（比如{ (*/) factor }*）就会被清理掉。在一个成功的解析中，整个右边部分会完全展开来匹配输入令牌流。  
有了前面的知识背景，下面我们举例一个简单示例来展示如何构建一个递归下降表达式求值程序：

In [5]:
#! /usr/bin/env python
# -*- encoding: utf-8 -*-
"""
Topic: 下降解析器
Desc:
"""
import re
import collections

# Token specification
NUM = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
MINUS = r'(?P<MINUS>-)'
TIMES = r'(?P<TIMES>\*)'
DIVIDE = r'(?P<DIVIDE>/)'
LPAREN = r'(?P<LPAREN>\()'
RPAREN = r'(?P<RPAREN>\))'
WS = r'(?P<WS>\s+)'

master_pat = re.compile('|'.join([NUM, PLUS, MINUS, TIMES,
                                 DIVIDE, LPAREN, RPAREN, WS]))
# Tokenizer
Token = collections.namedtuple('Token', ['type', 'value'])

def generate_tokens(text):
    scanner = master_pat.scanner(text)
    for m in iter(scanner.match, None):
        tok = Token(m.lastgroup, m.group())
        if tok.type != 'WS':
            yield tok
            
# Parser
class ExpressionEvaluator:
    def parse(self, text):
        self.tokens = generate_tokens(text)
        self.tok = None # Last symbol consumed
        self.nexttok = None # Next symbol tokenized
        self._advance() # Load first lookahead token
        return self.expr()
    
    def _advance(self):
        '''Advance one token ahead'''
        self.tok, self.nexttok = self.nexttok, next(self.tokens, None)
        
    def _accept(self, toktype):
        '''Test and consume the next token if it matchs toktype'''
        if self.nexttok and self.nexttok.type == toktype:
            self._advance()
            return True
        else:
            return False
    
    def _expect(self, toktype):
        '''Column next token if it matchs toktype ot raise SynataxError'''
        if not self._accept(toktype):
            raise SyntaxError('Expected ' + toktype)
            
    # Grammar rules follow
    def expr(self):
        """expression ::= term { ('+'|'-') term }*"""
        exprval = self.term()
        while self._accept('PLUS') or self._accept('MINUS'):
            op = self.tok.type
            right = self.term()
            if op == 'PLUS':
                exprval += right
            elif op == 'MINUS':
                exprval -= right
        return exprval
        
    def term(self):
        """term : factor { ('*'|'/') factor }*"""
        termval = self.factor()
        while self._accept('TIMES') or self._accept('DIVIDE'):
            op = self.tok.type
            right = self.factor()
            if op == 'TIMES':
                termval *= right
            elif op == 'DIVIDE':
                termval /= right
        return termval
        
    def factor(self):
        """factor ::= NUM | ( expr )"""
        if self._accept('NUM'):
            return int(self.tok.value)
        elif self._accept('LPAREN'):
            exprval = self.expr()
            self._expect('RPAREN')
            return exprval
        else:
            raise SyntaxError('Expected NUMBER or LPAREN')

def descent_parser():
    e = ExpressionEvaluator()
    print(e.parse('2'))
    print(e.parse('2 + 3'))
    print(e.parse('2 + 3 * 4'))
    print(e.parse('2 + (3 + 4) * 5'))
    
if __name__ == '__main__':
    descent_parser()

2
5
14
37


##### 讨论
文本解析是一个很大的主题，一般会占用学生学习编译课程时刚开始的三周时间。如果你在找寻关于语法，解析算法等相关的背景知识的话，你应该去看一下编译器书籍。很显然，关于这方面的内容太多，不可能在这里全部展开。  
尽管如此，编写一个递归下降解析器的整体思路是比较简单的。开始时，你先获得所有语法规则，然后将其转换为一个函数或者方案。因此如果你语句类似这样：

你应该首先将它们转换成一组像下面这样的方法：

In [6]:
class ExpressEvaluator:
    def expr(self):
        pass
    def term(self):
        pass
    def factor(self):
        pass

每个方法要完成的任务很简单，它必须从左至右遍历语法规则的每一部分，处理每个令牌。从某种意义上讲，方法的目的就是要么处理语法规则，要么产生一个语法错误。从这样做，需采用下面的这些实现方法：  
* 如果规则中的下个符号是另外一个语法规则的名字（比如term或factor），就简单的调用同名的方法即可。这就是该算法中“下降”的由来——控制下降到另一个语法规则中去。有时候规则会调用已经执行的方法（比如，在`factor ::= '('exper')'中对expr的调用`）。这就是算法中“递归”的由来。  
* 如果规则中下一个符号是个特殊符号（比如()，你得查找下一个令牌并确认是一个精确匹配）。如果不匹配，就产生一个语法错误。这一节中的_expert()方法就是用来做这一步的。  
* 如果规则中下一个符号为一些可能的选择项（比如+或-），你必须对每一种可能情况检查下一个令牌，只有当它匹配一个的时候才能继续。这也是本节实例中_accept()方法的目的。它相当于_expert()方法的弱化版本，因为如果一个匹配找到它会继续，但是如果没找到，它不会产生错误而是回滚（允许后续的检查继续进行）。
* 对于有重复部分的规则（比如在规则表达式`::= term { ('+'|'-') term }*`），重复动作通过一个while循环来实现。循环主题会收集或处理所有的重复元素直到没有其他元素可以找到。  
* 一旦整个语法规则处理完成，每个方法会返回某种结果给调用者。这就是在解析过程总值是怎样累加的原理。比如，在表达式求值程序中，返回值代表表达式解析后的部分结果。最后所有值会在最顶层的语法规则方法中合并起来。  
尽管像你演示的是一个简单的例子，递归下降解析器可以用来实现非常负载的解析。比如，python语言本身就是一个递归下降解析器去解释的。可以通过查看python源码文件Grammar/Grammar来研究底层语法机制。看完你会发现，通过手动方式趋势心啊一个解析器其实会有很多的局限和不足之处。